In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymystem3
import re
from numba import jit
import numpy as np

In [2]:
from multiprocessing import Pool

## Описание полей
 * __id__ &mdash; внутренний идетификатор
 * __name__ &mdash; название вакансии
 * __description__ &mdash; текст вакансии
 * __target__ &mdash; класс заинтересованности

In [4]:
def get_data(processed=False):
    if processed:
        train_data = pd.read_csv('data/train_processed.csv', sep='\t', index_col='id')
        test_data = pd.read_csv('data/test_processed.csv', sep='\t', index_col='id') 
    else:
        train_data = pd.read_csv('data/train.csv', sep='\t', index_col='id')
        test_data = pd.read_csv('data/test.csv', sep='\t', index_col='id') 
    return (train_data, test_data)

In [5]:
## Токенизатор 
def get_lemmas(text):
    text = BeautifulSoup(text, 'lxml').get_text()       
    lemms = mystem.lemmatize(text)
    return " ".join([x for x in lemms if (re.match('^[a-zа-яё]+(-[a-zа-яё]+)*$', x) and len(x)>2)])
       
def get_vacancies_lemmas(texts):  
    texts = np.array(texts)    
    processed = np.vectorize(get_lemmas, otypes=[str])              
    return processed(texts)

In [ ]:
if __name__ == '__main__':
    mystem = pymystem3.Mystem()
    mystem.start()
    train_data = pd.read_csv('data/train.csv', sep='\t', index_col='id')   
    train_data['name_processed'] = get_vacancies_lemmas(train_data['name'])
    train_data['description_processed'] = get_vacancies_lemmas(train_data['description'])
    train_data.to_csv('data/train_processed2.csv', '\t', encoding='utf-8')
    mystem.close()

In [ ]:
if __name__ == '__main__':
    pool = Pool(2)
    mystem = pymystem3.Mystem()
    mystem.start()    
    X = pool.map(get_vacancies_lemmas, [train_data['name'][-10:], train_data['description'][-10:]])    
    mystem.close()
    print(X)